In [72]:
from PlainNet import FlairNPYSliceDataset, compute_pos_weight_from_dataset

path_data = "../01_data/04_flair_preproc_slices/"
train_ds = FlairNPYSliceDataset(root_dir=path_data, split="train")
val_ds   = FlairNPYSliceDataset(root_dir=path_data, split="val")
test_ds  = FlairNPYSliceDataset(root_dir=path_data, split="test")

print(len(train_ds), len(val_ds), len(test_ds))

# pos_weight 계산
print("Train pos_weight:", compute_pos_weight_from_dataset(train_ds))

# dataloader
from torch.utils.data import DataLoader
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0)
batch = next(iter(train_loader))
print(batch["image"].shape, batch["label"].shape)
print(batch["subject_id"])         # e.g. ['subject1', 'subject3', ...]
print(batch["slice_idx"])          # e.g. [12, 45, 3, 88, ...]
print(batch["label"])
# torch.Size([8,1,672,672]) torch.Size([8])

42435 9030 9048
Train pos_weight: tensor(48.4580)
torch.Size([8, 1, 672, 672]) torch.Size([8])
['BP00518_P16950', 'HP00094_P01876', 'HP00055_P05753', 'HP00421_P07283', 'HP00722_P07404', 'HP00334_P04870', 'HP00141_P01647', 'HP00770_P05887']
tensor([ 7, 20,  5, 16, 21, 20,  1, 19])
tensor([0., 0., 0., 0., 0., 0., 0., 0.])


In [138]:
import numpy as np
batch = next(iter(train_loader))
print(batch["image"].shape, batch["label"].shape)
print(batch["subject_id"])         # e.g. ['subject1', 'subject3', ...]
print(batch["slice_idx"])          # e.g. [12, 45, 3, 88, ...]
print(batch["label"])

idx = np.arange(8)[batch["label"] == 1]
print(np.array(batch["subject_id"])[idx])
print(batch['slice_idx'][idx])

torch.Size([8, 1, 672, 672]) torch.Size([8])
['HP00209_P02691', 'HP00729_P04570', 'HP00130_P04420', 'BP00185_P13256', 'BP00376_P11730', 'BP00418_P12627', 'BP00139_P12079', 'HP00878_P07632']
tensor([12, 13,  9,  7, 22, 22,  2,  4])
tensor([0., 1., 0., 0., 0., 0., 0., 0.])
['HP00729_P04570']
tensor([13])


In [1]:
import torch
from torchsummary import summary
from PlainNet.models.plainnet import PlainNet

# 모델 생성
model = PlainNet(backbone="resnet18", in_channels=1, pretrained=False)

# GPU에 올리기 (CUDA 있으면)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# summary 출력
summary(model, input_size=(1, 672, 672))  # (채널, H, W)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 336, 336]           3,136
       BatchNorm2d-2         [-1, 64, 336, 336]             128
              ReLU-3         [-1, 64, 336, 336]               0
         MaxPool2d-4         [-1, 64, 168, 168]               0
            Conv2d-5         [-1, 64, 168, 168]          36,864
       BatchNorm2d-6         [-1, 64, 168, 168]             128
              ReLU-7         [-1, 64, 168, 168]               0
            Conv2d-8         [-1, 64, 168, 168]          36,864
       BatchNorm2d-9         [-1, 64, 168, 168]             128
             ReLU-10         [-1, 64, 168, 168]               0
       BasicBlock-11         [-1, 64, 168, 168]               0
           Conv2d-12         [-1, 64, 168, 168]          36,864
      BatchNorm2d-13         [-1, 64, 168, 168]             128
             ReLU-14         [-1, 64, 1